<a href="https://colab.research.google.com/github/kimim9900/SImilar-materials-for-somescreen/blob/main/first.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install --upgrade git+https://github.com/qzhu2017/PyXtal.git@master


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/qzhu2017/PyXtal.git (to revision master) to /tmp/pip-req-build-t4t7ggyz
  Running command git clone --filter=blob:none --quiet https://github.com/qzhu2017/PyXtal.git /tmp/pip-req-build-t4t7ggyz
  Resolved https://github.com/qzhu2017/PyXtal.git to commit 6ccdc0d8bbc8d5c36fd92c6d84649543b8f50cb0
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.3/515.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4

In [2]:
!pip install rdkit-pypi


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 46.8 MB/s eta 0:00:00


In [ ]:
# I download the file for Organic-1.db, 2, 3, and 4
# and I also downloaded mech.db

In [3]:
import os
import sqlite3
import pandas as pd
from pyxtal.db import database

def check_table_exists(db_conn, table_name):
    cursr = db_conn.cursor()
    str = "select name from sqlite_master where type='table'"
    table_names = cursr.execute(str)
    print("Tables in the database:")
    tables=[]
    for name in table_names:
        tables.append(name[0])
    if table_name in tables:
        return True
    else:
        return False

# 1) Compute the template descriptor
mech_db = database('mech.db')
template_code = 'DAHMUX'
template_xtal = mech_db.get_pyxtal(template_code)
template_sph = template_xtal.get_spherical_images()

# Paths to the Organic database files
path_to_dbs = ['Organic-1.db', 'Organic-2.db', 'Organic-3.db', 'Organic-4.db']

results = []

# Loop over all database files
for path_to_db in path_to_dbs:

    # Create a connection to the Organic db
    conn = sqlite3.connect(path_to_db)

    if not check_table_exists(conn, 'systems'):
        print(f"'systems' table does not exist in the database {path_to_db}.")
        continue

    # Query to get all the unique_ids from 'systems' table in the current Organic db
    organic_codes_query = 'SELECT unique_id FROM systems'
    organic_codes = pd.read_sql_query(organic_codes_query, conn)['unique_id'].tolist()

    # Close the connection to the Organic db
    conn.close()

    # For each unique_id in organic db
    for unique_id in organic_codes:
        # Get the crystal structure and compute its descriptor
        organic_db = database(path_to_db)

        try:
            print(f"Details for unique_id {unique_id}:")
            xtal = organic_db.get_pyxtal(unique_id)
            print(xtal)
        except Exception as e:
            print(f"Failed to get details for unique_id {unique_id} due to error: {str(e)}")

        try:
            sph = xtal.get_spherical_images()
        except Exception as e:
            print(f"Failed to get spherical images for unique_id {unique_id} due to error: {str(e)}")
            continue

        # Compute the similarity (S) between sph1 and sph2
        try:
            S = template_sph.get_similarity(sph, M=10, cutoff=0.95).mean()
        except Exception as e:
            print(f"Failed to compute similarity for unique_id {unique_id} due to error: {str(e)}")
            continue

        # Save the information about (unique_id, S)
        results.append((unique_id, S))

# Export the (unique_id, S) to the csv file.
df = pd.DataFrame(results, columns=['unique_id', 'similarity'])
df.to_csv('similarity.csv', index=False)


Tables in the database:
Details for unique_id 1021e07653b9660d014c758778ebd19e:
Failed to get details for unique_id 1021e07653b9660d014c758778ebd19e due to error: cannot find the entry from 1021e07653b9660d014c758778ebd19e
Failed to get spherical images for unique_id 1021e07653b9660d014c758778ebd19e due to error: name 'xtal' is not defined
Details for unique_id 1022379ae32b33f912b010ea1823fff9:
Failed to get details for unique_id 1022379ae32b33f912b010ea1823fff9 due to error: cannot find the entry from 1022379ae32b33f912b010ea1823fff9
Failed to get spherical images for unique_id 1022379ae32b33f912b010ea1823fff9 due to error: name 'xtal' is not defined
Details for unique_id 1022d64f2007ce0d30f7ccda9bc7bb5d:
Failed to get details for unique_id 1022d64f2007ce0d30f7ccda9bc7bb5d due to error: cannot find the entry from 1022d64f2007ce0d30f7ccda9bc7bb5d
Failed to get spherical images for unique_id 1022d64f2007ce0d30f7ccda9bc7bb5d due to error: name 'xtal' is not defined
Details for unique_id 

KeyboardInterrupt: ignored